In [16]:
from datasets import load_dataset

ds = load_dataset("christopher/birdclef-2025")

In [12]:
!mkdir birdclef-2025

In [28]:
import os
import soundfile as sf
from tqdm import tqdm
from multiprocess import Pool
import itertools

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

def loop(indices):
    indices, _ = indices
    ds = load_dataset("christopher/birdclef-2025")
    data = []
    for i in tqdm(indices):
        row = ds['train'][i]
        
        if row['class_name'] == 'Aves':
            continue
            
        y = row['recording']['array']
        sr = row['recording']['sampling_rate']
        if (len(y) / sr) >= 30:
            continue
        
        audio_filename = os.path.join('birdclef-2025', row['recording']['path']).replace('.ogg', '.mp3')
        sf.write(audio_filename, y, sr)
            
        row.pop('recording')
        row['audio_filename'] = audio_filename
        row['len'] = len(y) / sr
        data.append(row)
        
    return data

In [29]:
processed = multiprocessing(list(range(len(ds['train']))), loop, cores = 20)

 95%|███████████████████████████████████████████████████████████████████████████████    | 1361/1428 [00:29<00:01, 53.58it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

 75%|██████████████████████████████████████████████████████████████▏                    | 1069/1428 [00:33<00:15, 23.44it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 1428/1428 [02:03<00:00, 11.60it/s]


In [32]:
len(processed)

573

In [31]:
processed[-1]

{'primary_label': '868458',
 'secondary_labels': [''],
 'type': [''],
 'collection': 'CSA',
 'rating': 0.0,
 'url': 'http://colecciones.humboldt.org.co/rec/sonidos/IAvH-CSA-34220/IAvH-CSA-34220.mp3',
 'latitude': -3.8333,
 'longitude': -70.3333,
 'scientific_name': 'Typophyllum inflatum',
 'common_name': 'Typophyllum inflatum',
 'author': 'Fernando Montealegre-Z',
 'license': 'cc-by-nc-sa 4.0',
 'inat_taxon_id': 868458,
 'class_name': 'Insecta',
 'audio_filename': 'birdclef-2025/CSA34220.mp3',
 'len': 2.924875}

In [34]:
import json

with open('BirdCLEF-2025.json', 'w') as fopen:
    json.dump(processed, fopen)